# Python library for the Apogee μCache AT-100 datalogger

Python script to connect to the [Apogee μCache AT-100 datalogger](https://www.apogeeinstruments.com/microcache-bluetooth-micro-logger/), makes precision environmental measurements using Apogee’s analog sensors. This script is meant for periodical data collection, while the μCache does internal datalogging. It also verifies the battery charge and alerts the user if the charge gets too low.

### Dependencies
- Bleak (v. 0.16, available in conda-forge. Note: Current version is 0.19)

In [ ]:
import asyncio
from bleak import BleakClient, BleakScanner

In [ ]:
async def scan_apogee_devices(duration=5):
    devices = {}
    async with BleakScanner() as scanner:
        await asyncio.sleep(duration)
        devices = scanner.discovered_devices
        
        apogee_devices = []
        for device in devices:
            metadata = device.metadata['manufacturer_data']
            if metadata:
                manufacturer_data = [(k,v) for k,v in device.metadata['manufacturer_data'].items()][0]
                company_id      = manufacturer_data[0]
                advertised_data = manufacturer_data[1]
            else:
                manufacturer_sig = None
                advertised_data = None
            
            if(company_id == 0x0644):
                advertised_data = advertised_data.decode('utf-8')
                apogee_devices.append({'address': device.address, 'name': device.name, 'rssi': device.rssi,
                                    'company_id': company_id, 'advertised_data': advertised_data})
    return(apogee_devices)

async def connect_to_device(address):
    client = BleakClient(address)
    await client.connect()
    return client

async def read_device_info(client):
    dis_service = "0000180A-0000-1000-8000-00805f9b34fb"
    dis_characteristics = {
        "manufacturer_name": "00002A29-0000-1000-8000-00805f9b34fb",
        "model_number": "00002A24-0000-1000-8000-00805f9b34fb",
        "serial_number": "00002A25-0000-1000-8000-00805f9b34fb",
        "firmware_revision": "00002A26-0000-1000-8000-00805f9b34fb",
        "hardware_revision": "00002A27-0000-1000-8000-00805f9b34fb",
    }
    device_info = {}
    for characteristic in dis_characteristics:
        uuid = dis_characteristics[characteristic]
        value = await client.read_gatt_char(uuid)
        value_str = value.decode("utf-8").strip() if value else ""
        device_info.update({characteristic: value_str})
    return device_info

async def read_battery_level(client):
    battery_service = "0000180F-0000-1000-8000-00805f9b34fb"
    battery_level = "00002A19-0000-1000-8000-00805f9b34fb"
    battery_value = await client.read_gatt_char(battery_level)
    return int.from_bytes(battery_value, byteorder='little')

async def disconnect_from_device(client):
    await client.disconnect()

In [ ]:
async def run():
    print('Searching for devices:')
    device_list = await scan_apogee_devices(2)
    if(len(device_list) == 0):
        print('  - Device not found, make sure it is broadcasting through a long press')
        print('Done...')
        return
    device_address = device_list[0]['address']
    print('  - Found device at address', device_address)
    
    print('  - Connecting to device')
    client = await connect_to_device(device_address)
    
    print('  - Reading device information:')
    device_info = await read_device_info(client)
    print('    Type:  ', device_info['manufacturer_name'], device_info['model_number'])
    print('    Serial:', device_info['serial_number'])
    print('    Firmware rev.:', device_info['firmware_revision'])
    print('    Hardware rev.:', device_info['hardware_revision'])
    
    print('  - Reading battery level:')
    battery_level = await read_battery_level(client)
    print('    Level: ' + str(battery_level) + '%')
    
    print('  - Disconnecting')
    await disconnect_from_device(client)
    print('Done...')
    pass

await run()

In [ ]:
device_list = await scan_apogee_devices(3)
print(device_list)

## OLD

In [ ]:
# OLD
async def read_device_info(address):
    client = bleak.BleakClient(address)
    try:
        if not client.is_connected:
            await client.connect(timeout=5.0)
    except bleak.exc.BleakError as e:
        print(f"Connection error: {e}")
        return {}

    try:
        # Get the Device Information Service by UUID
        services = await client.get_services()
        dis_service = next((s for s in services if s.uuid == "0000180a-0000-1000-8000-00805f9b34fb"), None)
        if dis_service is None:
            print("Device Information Service not found")
            return {}

        # Read the desired characteristics from the service
        manufacturer_name = await client.read_gatt_char("00002a29-0000-1000-8000-00805f9b34fb", service=dis_service)
        model_number = await client.read_gatt_char("00002a24-0000-1000-8000-00805f9b34fb", service=dis_service)
        serial_number = await client.read_gatt_char("00002a25-0000-1000-8000-00805f9b34fb", service=dis_service)
        firmware_revision = await client.read_gatt_char("00002a26-0000-1000-8000-00805f9b34fb", service=dis_service)
        hardware_revision = await client.read_gatt_char("00002a27-0000-1000-8000-00805f9b34fb", service=dis_service)
        
        return {
            "Manufacturer Name": manufacturer_name.decode(),
            "Model Number": model_number.decode(),
            "Serial Number": serial_number.decode(),
            "Firmware Revision": firmware_revision.decode(),
            "Hardware Revision": hardware_revision.decode(),
        }
    except:
        print("Device not found. Please activate it using a long press on the button")
        return {}
    finally:
        await client.disconnect()
        
async def read_battery_level(address):
    client = bleak.BleakClient(address)
    try:
        if not client.is_connected:
            await client.connect(timeout=5.0)
    except bleak.exc.BleakError as e:
        print(f"Connection error: {e}")
        return {}

    try:
        # Get the Services by UUID
        services = await client.get_services()
        
        bat_service = next((s for s in services if s.uuid == "0000180f-0000-1000-8000-00805f9b34fb"), None)
        if bat_service is None:
            print("Device Information Service not found")
            return {}
        battery_level = await client.read_gatt_char("00002a19-0000-1000-8000-00805f9b34fb", service=dis_service)

        return battery_level
    except:
        print("Device not found. Please activate it using a long press on the button")
        return {}
    finally:
        await client.disconnect()
        
async def main():
    print('Searching for devices:')
    device_list = await scan_apogee_devices(2)
    address = device_list[0]['address']
    print('  - Found device at address', address)
    
    print('  - Reading battery level:')
    bat_perc = await read_battery_level(address)
    print(bat_perc)
    bat_perc = int.from_bytes(bat_perc, byteorder="little")
    print('    Level: ' + str(bat_perc) + '%')
    
    print('  - Reading device information:')
    device_info = await read_device_info(address)
    print('    Type:  ', device_info['Manufacturer Name'], device_info['Model Number'])
    print('    Serial:', device_info['Serial Number'])
    print('    Firmware rev.:', device_info['Firmware Revision'])
    print('    Hardware rev.:', device_info['Hardware Revision'])
    pass

await main()

In [ ]:
# Further tools
'''
This function takes a socket sock as its input parameter and returns a dictionary with the values of the DIS characteristics. You can call this function by passing in the socket returned by ble.connect(). For example:

sock = ble.connect("AA:BB:CC:DD:EE:FF")
dis_characteristics = read_dis_characteristics(sock)
print(dis_characteristics)

Access specific characteristics like this:
print(dis_characteristics["manufacturer_name"])
'''
def read_dis_characteristics(sock):
    # Get the handles for the DIS characteristics
    dis_service_uuid = ble.UUID("0000180a-0000-1000-8000-00805f9b34fb")
    handles = ble.get_service_handles(sock, dis_service_uuid)

    # Read the Manufacturer Name characteristic
    manufacturer_name_uuid = ble.UUID("00002a29-0000-1000-8000-00805f9b34fb")
    manufacturer_name = ble.read_characteristic(sock, handles, manufacturer_name_uuid)

    # Read the Model Number characteristic
    model_number_uuid = ble.UUID("00002a24-0000-1000-8000-00805f9b34fb")
    model_number = ble.read_characteristic(sock, handles, model_number_uuid)

    # Read the Serial Number characteristic
    serial_number_uuid = ble.UUID("00002a25-0000-1000-8000-00805f9b34fb")
    serial_number = ble.read_characteristic(sock, handles, serial_number_uuid)

    # Read the Firmware Revision characteristic
    firmware_revision_uuid = ble.UUID("00002a26-0000-1000-8000-00805f9b34fb")
    firmware_revision = ble.read_characteristic(sock, handles, firmware_revision_uuid)

    # Read the Hardware Revision characteristic
    hardware_revision_uuid = ble.UUID("00002a27-0000-1000-8000-00805f9b34fb")
    hardware_revision = ble.read_characteristic(sock, handles, hardware_revision_uuid)

    # Create a dictionary with the characteristic values
    characteristics = {
        "manufacturer_name": manufacturer_name.decode(),
        "model_number": model_number.decode(),
        "serial_number": serial_number.decode(),
        "firmware_revision": firmware_revision.decode(),
        "hardware_revision": hardware_revision.decode()
    }

    return characteristics

'''
Read the Battery Level characteristic from a Bluetooth LE device and returns its value as a percentage
'''
def read_battery_level(sock):
    # Get the handle for the Battery Level characteristic
    battery_service_uuid = ble.UUID("0000180f-0000-1000-8000-00805f9b34fb")
    handles = ble.get_service_handles(sock, battery_service_uuid)

    battery_level_uuid = ble.UUID("00002a19-0000-1000-8000-00805f9b34fb")
    battery_level = ble.read_characteristic(sock, handles, battery_level_uuid)

    # Return the battery level value as a percentage
    return ord(battery_level)
    
'''
The Live Data Control Characteristic controls averaging time of the Live Data Characteristic. Valid values are 0-127 in units of 0.25 seconds. A value of 0 is interpreted as no averaging with data calculated from a single ADC sample.
'''
def set_averaging_time_apogee(sock, value_sec):
    # Convert the value from seconds to units of 0.25 seconds
    time_units = int(value_sec / 0.25)
    time_units = min(max(time_units, 0), 127)  # Clamp the value to the valid range

    # Write the value to the characteristic
    base_uuid = ble.UUID("b3e00000-2594-42a1-a5fe-4e660ff2868f")
    service_uuid = ble.UUID("0005", base_uuid)
    handle = ble.get_characteristic_handle(sock, service_uuid)
    ble.write_command(sock, handle, time_units.to_bytes(1, 'little'))
    pass
    
'''
Check and set time
'''
import time
import struct
from datetime import datetime
from ntplib import NTPClient
from bluepy.btle import Peripheral, UUID

def set_current_time(ntp_server='pool.ntp.org', timeout=5):
    # Define the UUIDs for the device and service
    base_uuid = UUID("b3e0xxxx-2594-2a1a-5fe4-e660ff2868f")
    service_uuid = UUID("000a")

    # Create a connection to the device
    device = Peripheral('AA:BB:CC:DD:EE:FF')
    service = device.getServiceByUUID(UUID(str(base_uuid).replace('xxxx', str(service_uuid))))

    # Read the current Unix Epoch time from the device
    characteristic = service.getCharacteristics(UUID("0002"))[0]
    device_time = struct.unpack("<I", characteristic.read())[0]

    # Read the current time from the NTP server
    ntp_client = NTPClient()
    ntp_response = ntp_client.request(ntp_server, version=3)
    ntp_time = int(ntp_response.tx_time)

    # Calculate the difference between the device time and the NTP time
    time_diff = abs(device_time - ntp_time)

    # If the difference is greater than 3 seconds, update the device time
    if time_diff > 3:
        # Convert the NTP time to an unsigned 32-bit integer
        ntp_time_bytes = struct.pack("<I", ntp_time)

        # Write the NTP time to the device
        characteristic = service.getCharacteristics(UUID("0001"))[0]
        characteristic.write(ntp_time_bytes, withResponse=True)

    # Disconnect from the device
    device.disconnect()

    # Return the current Unix Epoch time
    return datetime.fromtimestamp(device_time).strftime("%Y-%m-%d %H:%M:%S")